<a href="https://colab.research.google.com/github/ravi02512/Sentiment-classification-using-LSTM-Network/blob/master/Sentiment%20classifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)



import pandas as pd 
import numpy as np

Using TensorFlow backend.


In [0]:
#helper functions


import numpy as np
import pandas as pd

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

    

In [0]:
train = pd.read_csv('/content/drive/My Drive/sentiment/train_2kmZucJ.csv')
test= pd.read_csv('/content/drive/My Drive/sentiment/test_oJQbWVk.csv')

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/sentiment/glove.twitter.27B.200d.txt')

dictionary=word_to_vec_map.keys()

In [0]:
import re
clean_train=[]
for i in range(len(train)):
  new=re.sub('[^A-Za-z ]+', ' ', train['tweet'][i])
  clean_train.append(new)

In [0]:
tokens=[]
for i in range(len(clean_train)):
  tokens.extend(clean_train[i].split(' '))

In [0]:
count_df=pd.DataFrame(tokens,columns=['words'])
count_df=count_df['words'].value_counts().reset_index().rename(columns={'index':'words', 'words':'counts'})
count_df['word length']=[len(w) for w in count_df['words']]
count_df.head()

,words,counts,word length
0,,75832,0
1,http,3442,4
2,com,3389,3
3,iphone,2798,6
4,p,2724,1


In [0]:
g=[]
for w in count_df['words']:
  if w in dictionary:
    g.append(w)
    
print('number of words belong to dictionary'len(g))

SyntaxError: ignored

In [0]:
length=[]
for i in range(len(clean_train)):
  length.append(len([word.lower() for word in clean_train[i].split()]))

In [0]:
max_len=np.max(length)
print(max_len)

59


In [0]:

def sentences_to_indices(X, word_to_index, max_len):
    """
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                        
    
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                                     
        sentence_words = [word.lower() for word in X[i].split()]
        j = 0
        for w in sentence_words:
          if w.lower() in dictionary:
              X_indices[i, j] = word_to_index[w.lower()]
              #
              j = j + 1


    
    return X_indices

1193515

In [0]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                 
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    
    for word, index in word_to_index.items():
      if index!= 1800:
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    

    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:

def my_model(input_shape, word_to_vec_map, word_to_index):
    """
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    sentence_indices = Input(input_shape, dtype = 'int32')
    
  
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)
    
    X = LSTM(128, return_sequences = True)(embeddings)
    X = Dropout(0.5)(X)
    
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X)
    
    
    return model

In [0]:
import gc 
gc.collect()

430

In [0]:
model = Emojify_V2((max_len,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 59)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 59, 200)           238703000 
_________________________________________________________________
lstm_7 (LSTM)                (None, 59, 128)           168448    
_________________________________________________________________
dropout_7 (Dropout)          (None, 59, 128)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258 

In [0]:
from keras import optimizers
# sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
xtrain=np.asarray(clean_train)
y_train=np.asarray(ytrain)

In [0]:
X_train_indices = sentences_to_indices(xtrain, word_to_index, max_len)
Y_train_oh = convert_to_one_hot(y_train, C = 2)


In [0]:
model.fit(X_train_indices, Y_train_oh,validation_split=0.1, epochs = 50, batch_size = 32, shuffle=True)

Train on 7128 samples, validate on 792 samples
Epoch 1/50
7128/7128 [==============================] - 30s 4ms/step - loss: 0.3339 - acc: 0.8530 - val_loss: 0.2810 - val_acc: 0.8889
Epoch 2/50
7128/7128 [==============================] - 27s 4ms/step - loss: 0.2627 - acc: 0.8890 - val_loss: 0.2485 - val_acc: 0.8977
Epoch 3/50
7128/7128 [==============================] - 29s 4ms/step - loss: 0.2349 - acc: 0.9031 - val_loss: 0.3028 - val_acc: 0.8422
Epoch 4/50
7128/7128 [==============================] - 30s 4ms/step - loss: 0.3018 - acc: 0.8483 - val_loss: 0.2679 - val_acc: 0.8902
Epoch 5/50
7128/7128 [==============================] - 29s 4ms/step - loss: 0.2492 - acc: 0.9026 - val_loss: 0.2831 - val_acc: 0.8851
Epoch 6/50
7128/7128 [==============================] - 29s 4ms/step - loss: 0.2661 - acc: 0.8847 - val_loss: 0.2870 - val_acc: 0.8826
Epoch 7/50
7128/7128 [==============================] - 29s 4ms/step - loss: 0.2771 - acc: 0.8879 - val_loss: 0.3240 - val_acc: 0.8384
Epoch 8/

In [0]:
test_clean=[]
for i in range(len(test)):
  new=re.sub('[^A-Za-z ]+', ' ', test['tweet'][i])
  test_clean.append(new)

In [0]:
xtest=np.asarray(test_clean)

In [0]:
xtest[1]

'currently shitting my fucking pants   apple  iMac  cashmoney  raddest  swagswagswag http instagr am p UUIS bIBZo '

In [0]:
X_test_indices = sentences_to_indices(xtest, word_to_index, max_len)

In [0]:
y_pred=model.predict(X_test_indices)

In [0]:
pred=[]
for i in range(len(y_pred)):
  pred.append(y_pred[i].tolist().index(y_pred[i].max()))

In [0]:
import os
sample_sub=pd.read_csv('/content/drive/My Drive/sentiment/sample_submission_LnhVWA4.csv')
sample_sub['label']=pred
os.chdir('/content/drive/My Drive/sentiment')
sample_sub.to_csv('sub14.csv', index=False)